### Initialisations

In [6]:
#from gensim.corpora.dictionary import Dictionary
import os
import numpy as np

# select the number of documents to parse
Subset = 100 #10769
filewithTFIDF = "TFIDF_"+str(Subset)+"docs.npy" # this is used later
Distances = np.load(str(Subset)+'New_Doc_Distances.npy')

N_Topics = 5

FolderToParse = "BoW_100random/"#"BagsOfWords/"
DocList = []
for document in os.listdir( FolderToParse ):
    # load documents
    FileToLoad = FolderToParse + document
    f = open(FileToLoad,'rb')
    words = f.read().decode('ascii', 'ignore')
    f.close()
    words = words.split() # tokenize
    DocList.append(words)
    
DocList = DocList[:Subset]
# Load the names of species
with open('list_of_species.txt', encoding='utf-8', errors='ignore') as f:
    Names = f.readlines()
Names = [x.strip() for x in Names]

# Create dictionary [ID to word]
common_dictionary = Dictionary(DocList)
# Create text to words mappings & count
common_corpus = [common_dictionary.doc2bow(text) for text in DocList]
print("We have loaded {0} documents and {1} words in total!".format(len(DocList), len(common_dictionary)))

We have loaded 100 documents and 4893 words in total!


### Other utilities

In [ ]:
def ComputeDistances(D, C1, C2):
    """
        Function that, given two sets of indices C1, C2 and a matrix D with  
        distances calculated for every pair, it computes the average distance.
    """
    S=0
    for i in range(len(C1)):
        for j in range(len(C2)):
            S += D[ C1[i], C2[j] ]
    return S/(len(C1)*len(C2))
     
def EvalClustering(D, Clustering):
    """
        Function that, given a set clusters and a matrix D with distances calculated 
        for every pair of points, evaluates the accuracy of the partition.
        Intra : the average distance for points within one cluster
        Inter : the average distance between points from different clusters.
    """
    N_K = len(Clustering)
    ClusterDist = np.zeros( (N_K,N_K) )
    for c1 in range(N_K):
        if len(Clustering[c1])>0:
            for c2 in range(c1,N_K):
                #first we compute the intra-cluster distance
                if len(Clustering[c2])>0:
                    ClusterDist[c1,c2] = ComputeDistances(D, list(Clustering[c1]), list(Clustering[c2]))
    # evaluate
    intra = np.mean(np.diag(ClusterDist))
    print('Mean Within-Cluster distance = {0:.3f}.'.format(intra))
    inter = np.sum(np.triu(ClusterDist,1))*2/(N_K-1)/N_K
    print('Mean Inter-Cluster distance = {0:.3f}.'.format(inter))
    return intra, inter, ClusterDist

### 1. LDA by gensim

In [ ]:
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=N_Topics, alpha='asymmetric', random_state=1)
# Now produce probabilities based on the Corpus
LDAvectors = []
for i in range(len(DocList)):
    # first we translate using the dictionary that we have already
    temp = [ common_dictionary.doc2bow(text.split()) for text in DocList[i] ]
    vector = lda[temp[0]]
    LDAvectors.append( vector )
print('LDA is complete!')

#### 1b. Evaluate - gensim LDA with BW

In [ ]:
LDAvectors = np.load("LDAvectors_gensim_"+str(Subset)+".npy")

# Clustering #
ClustersLDA = { i:[] for i in range(N_Topics)}# initialize the dictictionary of clusters
ClustersNames = { i:[] for i in range(N_Topics)} 
Labels = []
for i in range(len(DocList)):
    distr = [ x[1] for x in LDAvectors[i]]
    # find the argmax{distr} - ATTENTION: ties ???
    label = distr.index(max(distr))
    ClustersLDA[label].append(i)
    ClustersNames[label].append(Names[i])
    Labels.append( label )
    
# Evaluate
t = EvalClustering(Distances, ClustersLDA)

#### 1c. Evaluate - gensim LDA with KMeans

In [ ]:
from sklearn.cluster import KMeans
# load data (again)
X =  np.load("LDAvectors_gensim_"+str(Subset)+".npy")

# possibly needs transform
X = np.array([[P[1] for P in Z] for Z in X])
print("n_samples: %d, n_features: %d" % X.shape)

km = KMeans(n_clusters=N_Topics, init='k-means++', max_iter=100, n_init=1, random_state=10)
print("Clustering sparse data with %s" % km)
km.fit(X)

# create the clusters
Clusters = { i:[] for i in range(N_Topics)}
Labels = list(km.labels_)
for i in range(len(DocList)):
    Clusters[LabelsKM[i]].append(i)
    
# Evaluate
t = EvalClustering(Distances, Clusters)

### 2. LDA by Scikit-learn

In [75]:
from sklearn.decomposition import LatentDirichletAllocation

X = np.load(filewithTFIDF)

print("n_samples: %d, n_features: %d" % X.shape)
LDA_SKL = LatentDirichletAllocation(n_topics = N_Topics, max_iter=1000, random_state=2)
LDA_SKL.fit(X)
print("Clustering sparse data with %s" % LDA_SKL)
# get doc-topic distributions
LDA_SKLvectors = LDA_SKL.transform(X)

n_samples: 100, n_features: 4893


/auto/dtchome/kalantzisg/.local/lib/python3.4/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Clustering sparse data with LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=1000, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=5, perp_tol=0.1,
             random_state=2, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


#### 2b. Evaluate scikit LDA with BW

In [ ]:
# Clustering - Black and white approach, as before
ClustersSKL = { i:[] for i in range(N_Topics)}
LabelsSKL = []
for i in range(len(DocList)):
    distr = list(LDA_SKLvectors[i])
    # find the argmax{distr} - ATTENTION: ties ???
    label = distr.index(max(distr))
    ClustersSKL[label].append(i)
    LabelsSKL.append( label )
    
# Evaluate
t = EvalClustering(Distances, ClustersSKL)

#### 2c. Evaluate scikit LDA with K-Means

In [ ]:
X = LDA_SKLvectors

km = KMeans(n_clusters=N_Topics, init='k-means++', max_iter=100, n_init=1, random_state=10)
print("Clustering sparse data with %s" % km)
km.fit(X)

# create the clusters
Clusters = { i:[] for i in range(N_Topics)}
Labels = list(km.labels_)
for i in range(len(DocList)):
    Clusters[LabelsKM[i]].append(i)
    
# Evaluate
t = EvalClustering(Distances, Clusters)

### 3. Benchmark technique: K-Means on TFIDF

In [ ]:
from sklearn.cluster import KMeans

X =  np.load(filewithTFIDF)

print("n_samples: %d, n_features: %d" % X.shape)
km = KMeans(n_clusters=N_Topics, init='k-means++', max_iter=100, n_init=1, random_state=10)
print("Clustering sparse data with %s" % km)
km.fit(X)

# create the clusters
ClustersKM = { i:[] for i in range(N_Topics)}
LabelsKM = list(km.labels_)
for i in range(len(DocList)):
    ClustersKM[LabelsKM[i]].append(i)
    
# Evaluate
t = EvalClustering(Distances, ClustersKM)

#### 3b. Advanced approach: SVD + K-Means

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
# Dimensionality reduction
svd = TruncatedSVD(50)
# we use the same X as before
X = svd.fit_transform(X)
print("n_samples: %d, n_features: %d" % X.shape)
km = KMeans(n_clusters=N_Topics, init='k-means++', max_iter=100, n_init=1, random_state=10)
print("Clustering sparse data with %s" % km)
km.fit(X)

# create the clusters
ClustersKM = { i:[] for i in range(N_Topics)}
LabelsKM = list(km.labels_)
for i in range(len(DocList)):
    ClustersKM[LabelsKM[i]].append(i)
    
# Evaluate
t = EvalClustering(Distances, ClustersKM)

### 4. Our LDA Implementation

In [4]:
# Implement LDA
# Number matrix (Replacing words in documents with word IDs)
from time import time
N_K = 5 # N_Topics # set the number of topics
N_D = len(DocList)
corpus=np.unique(np.concatenate(DocList),axis=0)
N_W = corpus.shape[0] # words in the vocabulary

# SELECT #iterations
T=100

X_number = np.copy(DocList)
for doc_number in range(X_number.shape[0]):
    for doc_length in range(len(X_number[doc_number])):
        X_number[doc_number][doc_length]=  np.where(corpus==X_number[doc_number][doc_length])[0][0]
        
# Dirichlet priors
alpha = 1 # Choice of alpha affects document clustering 
gamma = 1

#Z = np.copy(X_number)
#for doc_number in range(Z.shape[0]):
#    for doc_length in range(len(Z[doc_number])):
#        Z[doc_number][doc_length]= np.random.randint(N_K)
        
Z = []#[np.array(N_D, dtype=object)]
for doc in range(N_D):
    Z.append( np.random.randint(N_K, size=len(DocList[doc])) )
        
# Pi := document topic distribution
Pi = np.zeros([N_D, N_K])
for i in range(N_D):
    Pi[i] = np.random.dirichlet(alpha*np.ones(N_K))

A = Pi
#print(A)

# B := word topic distribution
B = np.zeros([N_K, N_W])
for k in range(N_K):
    B[k] = np.random.dirichlet(gamma*np.ones(N_W))
t0 = time()    
print("Starting the big loop...")    
for iterations in range(T):  #Need at least 1000 iterations for Gibbs sampling to work!

    # Updating Z matrix
    for doc_number in range(N_D):     
        for doc_length in range(len(Z[doc_number])):     
            # Calculate params for Z
            p_iv = np.exp(np.log(Pi[doc_number]) + np.log( B[:, X_number[doc_number][ doc_length]] ))
            p_iv /= np.sum(p_iv)

             # Resample word topic assignment Z
            Z[doc_number][doc_length] = np.random.multinomial(1, p_iv).argmax()
    # Updating Pi   
    for i in range(N_D):
        # Gather sufficient statistics
        ###m = np.zeros(N_K)
        ###for k in range(N_K):
        ###    m[k] = np.sum(Z[i] == k)
        
        m = np.array( [np.sum(Z[i] == k) for k in range(N_K)] )
        # Resample doc topic dist.
        Pi[i, :] = np.random.dirichlet(alpha + m)
        
    #Updating B
    for k in range(N_K):
        #print(k)
        n = np.zeros(N_W) 
    
        #Gather statistics       
        for v in range(N_W):
            for doc_number in range(N_D):
                n[v] = len([ x for x in np.where(X_number[doc_number] == v) if Z[doc_number][x] ==k ])
                ###for doc_length in range(len(Z[doc_number])):
                ###    n[v] += (X_number[doc_number][ doc_length]==v) and (Z[doc_number][doc_length] ==k)
        
        # Resample word topic distribution
        B[k,:] = np.random.dirichlet(gamma+n)
    #progress check
    if (iterations-1)%10==0:
        print("More than {0} % is completed! Rate = {1}".format(100*iterations/T, (time()-t0)/iterations))
print('LDA is complete! Total time = {0}'.format(time()-t0))

Starting the big loop...
More than 1.0 % is completed! Rate = 7.754592418670654
More than 11.0 % is completed! Rate = 4.198843869295987
More than 21.0 % is completed! Rate = 4.0200606641315275
More than 31.0 % is completed! Rate = 3.9916513043065227
More than 41.0 % is completed! Rate = 4.006635694968991
More than 51.0 % is completed! Rate = 3.9947992773617016
More than 61.0 % is completed! Rate = 3.9865652029631566
More than 71.0 % is completed! Rate = 3.9873329552126604
More than 81.0 % is completed! Rate = 3.993535477438091
More than 91.0 % is completed! Rate = 3.9916497822646257
LDA is complete! Total time = 395.5250279903412


In [32]:
X

array([[ 0.1041913 ,  0.57478023,  0.06239164,  0.05222822,  0.04492622,
         0.03942166,  0.03511779,  0.03166205,  0.0288255 ,  0.02645543],
       [ 0.10418669,  0.57478482,  0.06239164,  0.05222822,  0.04492621,
         0.03942166,  0.03511779,  0.03166205,  0.0288255 ,  0.02645543],
       [ 0.10364865,  0.57532203,  0.06238137,  0.05223848,  0.04492728,
         0.03942145,  0.03511776,  0.03166203,  0.0288255 ,  0.02645543],
       [ 0.10364801,  0.57532275,  0.06238137,  0.05223848,  0.04492728,
         0.03942145,  0.03511776,  0.03166203,  0.0288255 ,  0.02645543],
       [ 0.10369829,  0.57527232,  0.06238145,  0.05223851,  0.04492728,
         0.03942146,  0.03511776,  0.03166203,  0.0288255 ,  0.02645543],
       [ 0.10425949,  0.57471192,  0.06239174,  0.05222824,  0.04492622,
         0.03942166,  0.03511779,  0.03166205,  0.0288255 ,  0.02645543],
       [ 0.60139871,  0.07765351,  0.06232877,  0.05221418,  0.04492401,
         0.0394202 ,  0.03511771,  0.03166202

#### 4b. Our LDA with BW

In [ ]:
# Clustering #
ClustersOUR = { i:[] for i in range(N_Topics)}# initialize the dictictionary of clusters
for i in range(len(DocList)):
    #distr = [ x[1] for x in Pi[i]]
    # find the argmax{distr} - ATTENTION: ties ???
    label = np.argmax(Pi[i])
    ClustersOUR[label].append(i)
    
# Evaluate
t = EvalClustering(Distances, ClustersOUR)

#### 4c. Our LDA with KMeans

In [ ]:
# create doc-word list of lists
"""
corpus=np.unique(np.concatenate(DocList))
TDM = np.zeros((len(common_dictionary),N_D)) # we start by a full matrix and then transform it
for doc in range(N_D):
    temp = np.unique( DocList[doc] ) # get the different words on this document
    #temp2 = np.zeros((N_V,1))
    for i in range( len(temp) ):
        word = temp[i]
        count = len([ x for x in DocList[doc] if x == word])
        # we must get the index of this word in the (total) corpus
        TDM[ np.where(corpus == word) , doc] = count

# sanity check
for doc in range(N_D):
    if sum(TDM[:,doc])!=len(DocList[doc]):
        print("Doc-{0} has a problem!".format(doc))
  
TDM = np.transpose(TDM)
"""
#print("n_samples: %d, n_features: %d" % TDM.shape)
#LDA_SKL = LatentDirichletAllocation(n_topics = 100, max_iter=50, random_state=1)
#LDA_SKL.fit( TDM )
# get doc-topic distributions
#LDA_SKLvectors = LDA_SKL.transform(TDM)
km = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1, random_state=1)
print("Clustering LDA data with %s" % km)
km.fit(LDA_SKLvectors)

# create the clusters
ClustersXX = { i:[] for i in range(N_Topics)}
LabelsXX = list(km.labels_)
for i in range(len(DocList)):
    ClustersXX[LabelsXX[i]].append(i)
    
# Evaluate
t = EvalClustering(Distances, ClustersXX)

### 6. Clustering with random assigning

In [ ]:
ClustersRAND = { i:[] for i in range(N_Topics)}# initialize the dictictionary of clusters
for i in range(len(DocList)):
    #distr = [ x[1] for x in Pi[i]]
    # find the argmax{distr} - ATTENTION: ties ???
    label = np.random.randint(N_Topics)
    ClustersRAND[label].append(i)
    
# Evaluate
t = EvalClustering(Distances, ClustersRAND)

## Comparison of LDA output

In [56]:
def KLdivergence(P,Q):
    #import numpy as np
    n = P.shape[0]
    total = []
    for subject in range(n):
        D = 0
        for x in range(P.shape[1]):
            D +=P[subject][x]*np.log( P[subject][x]/Q[subject][x] )
        total.append(D)
    return total #np.mean(total)

In [77]:
LDAour1 = Pi
LDAour2 = np.load("prob100it5topcristiana.npy")
#LDAskl1 = LDA_SKLvectors
LDAskl2 = LDA_SKLvectors

compare = [LDAour1, LDAour2, LDAskl1, LDAskl2]

DL_All = np.zeros((len(compare), len(compare)))
for i in range(len(compare)):
    for j in range(len(compare)):
        DL_All[i,j] = np.mean( KLdivergence(compare[i], compare[j]) )
DL_All

array([[ 0.        ,  0.5995593 ,  3.81734874,  3.81734874],
       [ 0.55178462,  0.        ,  3.92596919,  3.92596919],
       [ 1.71809235,  1.99350227,  0.        ,  0.        ],
       [ 1.71809235,  1.99350227,  0.        ,  0.        ]])

In [70]:
np.save('DifferentLDAs.npy',compare)

In [78]:
KLdivergence( LDAskl1, LDAour1 )

[1.8332762475876307,
 0.46417857818970443,
 0.49493944643137322,
 0.85725336640298355,
 2.0271776301668285,
 1.619456610414258,
 1.1390552004718135,
 1.1413990591956116,
 1.2523371176183093,
 1.3786597738381747,
 1.6651537017824045,
 2.0357819173612142,
 2.3227806576543037,
 1.413728245379569,
 1.1611388888170446,
 0.86955534820706182,
 1.5020122342098325,
 1.2934847461970278,
 1.3182752929474022,
 0.57924430728592835,
 1.6010915450205356,
 1.9549962297865895,
 1.5453323581689897,
 1.8846794253183872,
 0.82564885644874286,
 1.3335109102244529,
 3.2628100941175311,
 1.3204776509283704,
 1.2920303069752137,
 1.8250591354348302,
 1.755659665133551,
 0.67097760764485415,
 1.5255246438807981,
 0.94559553906258664,
 2.035798880377814,
 1.93839957561961,
 1.3180543046664752,
 1.3337994950828289,
 1.8535247565324635,
 2.1439874793996339,
 1.5392526859334792,
 1.3115418795763192,
 1.9605764363122764,
 3.339047979258229,
 1.5133244832575494,
 1.2943547395562074,
 1.4119664892660262,
 0.981480307

(100, 5)